In [13]:
import os, glob, math, random
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib
import trimesh
import plyfile
import open3d as o3d
from open3d_utils import make_pcd, make_line_set, make_mesh
import pyvista as pv
from pyvista import examples
pv.rcParams['transparent_background'] = True

# export DISPLAY for remote visualization
# os.environ['DISPLAY'] = ":77"

In [14]:
def read_plydata(mesh_ply_file):
    plydata = plyfile.PlyData.read(mesh_ply_file)
    mesh_faces = np.stack([list(item) for item in plydata['face']])[:,0]

    plydata_vertices_data = np.asarray(list(plydata['vertex']))
    mesh_vertices_data = []
    for line in plydata_vertices_data:
        mesh_vertices_data.append(np.asarray(list(line)))
    mesh_vertices_data = np.stack(mesh_vertices_data)
    mesh_vertices = mesh_vertices_data[:,:3]
    mesh_colors = mesh_vertices_data[:,3:]
    mesh_colors = (mesh_colors - mesh_colors.min(axis=0, keepdims=True)) / (mesh_colors.max(axis=0, keepdims=True) - mesh_colors.min(axis=0, keepdims=True))
    return mesh_vertices, mesh_faces, mesh_colors
        

def visualize_mesh(mesh_ply_file, deformations_as_color=False):
    if deformations_as_color:
        mesh_vertices, mesh_faces, mesh_colors = read_plydata(mesh_ply_file)
        o3d.visualization.draw_geometries([make_mesh(mesh_vertices, mesh_faces, mesh_colors)])
    else:
        mesh = o3d.io.read_triangle_mesh(mesh_ply_file)
        o3d.visualization.draw_geometries([make_mesh(mesh.vertices, mesh.triangles)])
        
    
def render_mesh_gif(mesh_ply_file, mesh_save_path, deformations_as_color=False):
    if deformations_as_color:
        mesh_vertices, mesh_faces, mesh_colors = read_plydata(mesh_ply_file)
    else:
        mesh = o3d.io.read_triangle_mesh(mesh_ply_file)
        mesh_vertices, mesh_faces, mesh_colors = np.asarray(mesh.vertices), np.asarray(mesh.triangles), None
        
    # change view to adjust the gif orientation
    viewup = [0, -8, 0] # pix3D_chairs
    p = pv.Plotter()
    trimesh_mesh = trimesh.Trimesh(mesh_vertices, faces=mesh_faces, process=False)
    pv_mesh = pv.wrap(trimesh_mesh)
    p.add_mesh(pv_mesh, scalars=mesh_colors, rgb=True, smooth_shading=True, cmap="rainbow")
    p.set_background("white")
    path = p.generate_orbital_path(factor=2.5, n_points=36, viewup=viewup, shift=0.2)
    p.open_gif("{}.gif".format(mesh_save_path))
    p.orbit_on_path(path, write_frames=True, viewup=viewup, step=0.05)
    p.close()

In [ ]:
# using the following cell to visualize 3D meshes with deformationas as color.
# need a display connect for open3d visualization
# on open3d windown, press Ctrl + B to undo backface culling.

# set output_dir to the data dump, and also provide the save directory.
# path to output dump (generated by running evaluate.sh) eg: output/shapenet/reconstruction_stage_two/car/dump/
output_dir = "./"
gif_save_dir = './'

input_image_files = sorted(glob.glob(os.path.join(output_dir, '*_image_input.png')))
colored_mesh_ply_files = sorted(glob.glob(os.path.join(output_dir, '*_color_coded_mesh.ply')))
non_colored_mesh_ply_files = sorted(glob.glob(os.path.join(output_dir, '*[0-9]_mesh.ply')))


for idx in range(0, len(input_image_files)):
    input_image = np.asarray(Image.open(input_image_files[idx]))
    plt.imshow(input_image[:,:,:3])
    plt.show()
    
    if len(colored_mesh_ply_files)>0:
        visualize_mesh(colored_mesh_ply_files[idx], deformations_as_color=True)
        render_mesh_gif(colored_mesh_ply_files[idx], mesh_save_path="{}/{}".format(gif_save_dir, str(idx).zfill(6)), deformations_as_color=True)
    elif len(non_colored_mesh_ply_files)>0:
        visualize_mesh(non_colored_mesh_ply_files[idx])
        render_mesh_gif(non_colored_mesh_ply_files[idx], mesh_save_path="{}/{}".format(gif_save_dir, str(idx).zfill(6)))